In [1]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim

from env_CatchPigs import EnvCatchPigs

In [2]:
class ReplayBuffer():
    def __init__(self, size, obs_dims):
        self.mem_size = size
        self.obs_mem = torch.zeros((self.mem_size, *obs_dims))
        self.act_mem = torch.zeros(self.mem_size, dtype=torch.int64)
        self.rew_mem = torch.zeros(self.mem_size, dtype=torch.float32)
        self.next_obs_mem = torch.zeros((self.mem_size, *obs_dims))
        self.done_mem = torch.zeros(self.mem_size, dtype=torch.bool)
        self.cntr = 0

    def push(self, obs, act, rew, next_obs, done):
        """
        obs :: torch tensor shape==(channels, height, width)
        act :: int
        rew :: int
        obs_ :: torch tensor shape==(channels, height, width)
        done :: bool
        """

        idx = self.cntr % self.mem_size
        self.obs_mem[idx] = obs
        self.act_mem[idx] = act
        self.rew_mem[idx] = rew
        self.next_obs_mem[idx] = next_obs
        self.done_mem[idx] = done 
        self.cntr += 1

    def sample(self, batch_size):
        max_idx = min(self.mem_size, self.cntr)
        idxs = np.random.choice(max_idx, batch_size, replace=False)
        obs_batch = self.obs_mem[idxs]
        act_batch = self.act_mem[idxs]
        rew_batch = self.rew_mem[idxs]
        next_obs_batch = self.next_obs_mem[idxs]
        done_batch = self.done_mem[idxs]

        return obs_batch, act_batch, rew_batch, next_obs_batch, done_batch

In [3]:
class DeepQNetwork(nn.Module):
    def __init__(self, obs_dims, num_acts, lr=1e-3):
        super(DeepQNetwork, self).__init__()
        
        self.conv1 = nn.Conv2d(obs_dims[0], 32, 4, stride=1)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1)
        self.conv3 = nn.Conv2d(64, 64, 2, stride=1)
        
        linear_input_dims = self._calc_conv_output_dims(obs_dims)
        
        self.linear1 = nn.Linear(linear_input_dims, 512)
        self.linear2 = nn.Linear(512, num_acts)
        
        self.optimizer = optim.RMSprop(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        
        
    def _calc_conv_output_dims(self, input_dims):
        tmp = torch.zeros((1, *input_dims))
        tmp = self.conv1(tmp)
        tmp = self.conv2(tmp)
        tmp = self.conv3(tmp)
        return int(np.prod(tmp.size()))
    
    
    def forward(self, obs):
        h = F.relu(self.conv1(obs))
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        # flatten conv layer output
        h = h.view(h.size()[0], -1)
        # conv_state shape is BS x (n_filters * H * W)
        h = F.relu(self.linear1(h))
        acts = self.linear2(h)

        return acts
        
        

In [4]:
class DQNAgent(object):
    def __init__(self, obs_dims, num_acts, gamma=0.99, epsilon=1, lr=0.01,
                 mem_size=10000, batch_size=32, eps_min=0.01, eps_dec=3e-4,
                 replace=100, chkpt_dir='tmp/dqn'):
        
        self.gamma = gamma
        self.epsilon = epsilon
        self.lr = lr
        self.num_acts = num_acts
        self.obs_dims = obs_dims
        self.batch_size = batch_size
        self.eps_min = eps_min
        self.eps_dec = eps_dec
        self.replace_target_cnt = replace
        self.chkpt_dir = chkpt_dir
        self.act_space = [i for i in range(num_acts)]
        self.learn_cnt = 0

        self.memory = ReplayBuffer(mem_size, obs_dims)
        self.q_eval = DeepQNetwork(self.obs_dims, self.num_acts, self.lr)
        self.q_next = DeepQNetwork(self.obs_dims, self.num_acts, self.lr)
        
        
    def choose_act(self, obs):
        """
        obs :: torch tensor shape==(3, 96, 96)
        """
        if np.random.random() > self.epsilon:
            acts = self.q_eval.forward(obs.unsqueeze(0))
            act = torch.argmax(acts).item()
        else:
            act = np.random.choice(self.act_space)

        return int(act)
    
    
    def store_transition(self, obs, act, rew, next_obs, done):
        self.memory.push(obs, act, rew, next_obs, done)
        
        
    def sample_memory(self):
        obs, act, rew, next_obs, done = self.memory.sample(self.batch_size)
        return obs, act, rew, next_obs, done
    
    
    def replace_target_network(self):
        if self.learn_cnt % self.replace_target_cnt == 0:
            self.q_next.load_state_dict(self.q_eval.state_dict())

            
    def decrement_epsilon(self):
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min
        
    
    def learn(self):
        if self.memory.cntr < self.batch_size:
            return

        self.q_eval.optimizer.zero_grad()

        self.replace_target_network()

        obs, act, rew, next_obs, done = self.sample_memory()
        indxs = np.arange(self.batch_size)

        q_pred = self.q_eval.forward(obs)[indxs, act]
        q_next = self.q_next.forward(next_obs).max(dim=1)[0]

        q_next[done] = 0.0
        q_target = rew + self.gamma * q_next

        loss = self.q_eval.loss(q_target, q_pred)
        loss.backward()
        self.q_eval.optimizer.step()
        self.learn_cnt += 1

        self.decrement_epsilon()

In [5]:
def train(max_iter, env, agent1, agent2):
    
    obs_list = env.get_obs()
    obs1 = torch.tensor(obs_list[0], dtype=torch.float).permute(2, 0, 1)
    obs2 = torch.tensor(obs_list[1], dtype=torch.float).permute(2, 0, 1)
    
    last_100_rew = [0 for i in range(100)]
    for i in range(max_iter):
        act1 = agent1.choose_act(obs1)
        act2 = agent2.choose_act(obs2)
        act_list = [act1, act2]
        # print("iter= ", i, env.agt1_pos, env.agt2_pos, env.pig_pos, env.agt1_ori, env.agt2_ori, 'action', act1, act2)
        env.render()
        rew_list, done = env.step(act_list)
        rew1 = rew_list[0]
        rew2 = rew_list[1]
        # print(rew1)
        _obs_list = env.get_obs()
        _obs1 = torch.tensor(_obs_list[0], dtype=torch.float).permute(2, 0, 1)
        _obs2 = torch.tensor(_obs_list[1], dtype=torch.float).permute(2, 0, 1)
        agent1.store_transition(obs1, act1, rew1, _obs1, done)
        agent2.store_transition(obs2, act2, rew2, _obs2, done)
        obs1 = _obs1
        obs2 = _obs2
        agent1.learn()
        agent2.learn()
        last_100_rew[i % 100] = rew1 + rew2
        
        #env.plot_scene()
        if rew1 + rew2 > 0:
            print("iter= ", i)
            print("Goal found!")

        if i % 100 == 0:
            print(f"Iter: {i}, Epsilon:{agent1.epsilon}, Reward: {sum(last_100_rew)}")

In [6]:
env = EnvCatchPigs(7, False)
max_iter = 5000

dqn1 = DQNAgent((3,21,21), 5)
dqn2 = DQNAgent((3,21,21), 5)

train(max_iter, env, dqn1, dqn2)

size of map should be an odd integer no smaller than 7
Iter: 0, Epsilon:1, Reward: 0
Iter: 100, Epsilon:0.9790000000000023, Reward: -580
Iter: 200, Epsilon:0.9490000000000056, Reward: -600
Iter: 300, Epsilon:0.9190000000000089, Reward: -580
Iter: 400, Epsilon:0.8890000000000122, Reward: -660
Iter: 500, Epsilon:0.8590000000000155, Reward: -400
Iter: 600, Epsilon:0.8290000000000188, Reward: -380
Iter: 700, Epsilon:0.7990000000000221, Reward: -460
Iter: 800, Epsilon:0.7690000000000254, Reward: -520
Iter: 900, Epsilon:0.7390000000000287, Reward: -360
Iter: 1000, Epsilon:0.709000000000032, Reward: -460
Iter: 1100, Epsilon:0.6790000000000354, Reward: -200
Iter: 1200, Epsilon:0.6490000000000387, Reward: -420
Iter: 1300, Epsilon:0.619000000000042, Reward: -220
Iter: 1400, Epsilon:0.5890000000000453, Reward: -160
Iter: 1500, Epsilon:0.5590000000000486, Reward: -240
Iter: 1600, Epsilon:0.5290000000000519, Reward: -400
Iter: 1700, Epsilon:0.49900000000005496, Reward: -220
Iter: 1800, Epsilon:0.46

KeyboardInterrupt: 

In [ ]:
dqn1.epsilon = 0.6
dqn2.epsilon = 0.6
train(10000, env, dqn1, dqn2)